In [1]:
import sys
sys.path.append('../../')
# from drums_data_gen_test import *
from drums_data_gen_multi_test import *
import Modules.Loaders.DataFormatter as DF
from Modules.Utils.Imports import *

Covasim 3.1.3 (2022-07-19) — © 2020-2022 by IDM


c:\Users\Teddy\anaconda3\envs\reu_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def to_torch(ndarray):
    arr = torch.tensor(ndarray, dtype=torch.float)
    arr.requires_grad_(True)
    arr = arr.to(device)
    return arr

def to_numpy(x):
    return x.detach().cpu().numpy()

device = torch.device(GetLowestGPU(pick_from=[0,1,2,3]))

Device set to cpu


In [3]:
population = int(50e3)
test_prob = 0.1
trace_lb = 0.0
trace_ub = 0.3
chi_type = 'piecewise'
keep_d = True
dynamic = True
masking = 3 # 0, 1, 2, 3
multiple = True
n_runs = 5
model_params = ModelParams(population, test_prob, trace_lb, trace_ub, chi_type, keep_d, dynamic, masking)

In [4]:
# drums_data_generator(model_params)
drums_data_generator_multi(model_params, n_runs)

Initializing sim with 50000 people for 182 days
  Running "Sim 0": 2020-02-01 ( 0/182) (0.57 s)  ———————————————————— 1%
  Running "Sim 0": 2020-02-11 (10/182) (0.84 s)  •——————————————————— 6%
  Running "Sim 0": 2020-02-21 (20/182) (1.18 s)  ••—————————————————— 11%
  Running "Sim 0": 2020-03-02 (30/182) (1.58 s)  •••————————————————— 17%
  Running "Sim 0": 2020-03-12 (40/182) (1.94 s)  ••••———————————————— 22%
  Running "Sim 0": 2020-03-22 (50/182) (2.32 s)  •••••——————————————— 28%
  Running "Sim 0": 2020-04-01 (60/182) (2.70 s)  ••••••—————————————— 33%
  Running "Sim 0": 2020-04-11 (70/182) (3.06 s)  •••••••————————————— 39%
  Running "Sim 0": 2020-04-21 (80/182) (3.38 s)  ••••••••———————————— 44%
  Running "Sim 0": 2020-05-01 (90/182) (3.70 s)  •••••••••——————————— 50%
  Running "Sim 0": 2020-05-11 (100/182) (3.95 s)  •••••••••••————————— 55%
  Running "Sim 0": 2020-05-21 (110/182) (4.21 s)  ••••••••••••———————— 61%
  Running "Sim 0": 2020-05-31 (120/182) (4.43 s)  •••••••••••••—

In [5]:
path = '../../Data/covasim_data/drums_data/tests/'
retrain = False
case_name = get_case_name(population, test_prob, trace_ub, keep_d, dynamic=dynamic, chi_type=chi_type)

if not masking==0:
    if masking==1:
        case_name = case_name + '_maskingthresh'
    elif masking==2:
        case_name = case_name + '_maskinguni'
    elif masking==3:
        case_name = case_name + '_maskingnorm'

if multiple:
    case_name = case_name + '_' + str(n_runs)
    params = DF.load_covasim_data(path, population, test_prob, trace_ub, keep_d, case_name, plot=True)
else:
    params = DF.load_covasim_data(path, population, test_prob, trace_ub, keep_d, case_name, plot=True)
    
# Example: 
# If the data generated has 50,000 agents, test_prob of 0.1, trace_ub of 0.3, is dynamic, includes diagnosed, chi type is piecewise, 
# includes adaptive masking, and we ran 200 simulations, 
# then the case_name variable would be equal to '50000_dynamic_piecewise_0.1_0.3_masking_200'

In [6]:
if multiple:
    data = np.mean(params['data'], axis=0)
    data = (data / params['population'])
else:
    data = params['data']
    data = (data / params['population']).to_numpy()

params.pop('data')
N = len(data)
t = np.arange(N)[:,None]

In [7]:
# rescale data values to represent number of people in state X rather than ratios
data *= population

plot=True
if plot:
    n = data.shape[1]
    col_names = list('STEAYDQRF') if keep_d else list('STEAYQRF')
    # plot compartments
    fig = plt.figure(figsize=(15, 15))
    
    if multiple:
        for i in range(1, n + 1):
            ax = fig.add_subplot(int(np.ceil(n / 3)), 3, i)
            ax.plot(t, data[:, i - 1], '.k', label='Covasim Data')
            ax.set_title(col_names[i - 1])
            ax.legend(fontsize=8)
            fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
            plt.tight_layout(pad=2)
            plt.savefig(os.path.join(path, case_name + '_avg' + '.png') )
            # plt.show()
    else:
        for i in range(1, n + 1):
            ax = fig.add_subplot(int(np.ceil(n / 3)), 3, i)
            ax.plot(t, data[:, i - 1], '.k', label='Covasim Data')
            ax.set_title(col_names[i - 1])
            ax.legend(fontsize=8)
            fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
            plt.tight_layout(pad=2)
            plt.savefig(os.path.join(path, case_name + '_single' + '.png') )
            # plt.show()